In [24]:
from vnstock3 import Vnstock
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
import statistics 
from datetime import datetime, timedelta
import copy


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [26]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [27]:
stock = Vnstock().stock(source='VCI')
vn30_tickers = stock.listing.symbols_by_group('VN30').values
vn30_tickers = np.delete(vn30_tickers, np.where(vn30_tickers == 'SSB')) # SSB chỉ có giá trị từ 2021 (quá ít data để sử dụng)
vn30_tickers

2024-12-12 13:06:06 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-12-12 13:06:06 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-12-12 13:06:06 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


array(['ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
       'HPG', 'MBB', 'MSN', 'MWG', 'PLX', 'POW', 'SAB', 'SHB', 'SSI',
       'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE'], dtype=object)

In [28]:
stock.listing.symbols_by_group('VN100').values

array(['AAA', 'ACB', 'ANV', 'ASM', 'BCG', 'BCM', 'BID', 'BMP', 'BSI',
       'BVH', 'BWE', 'CII', 'CMG', 'CRE', 'CTD', 'CTG', 'CTR', 'DBC',
       'DCM', 'DGC', 'DGW', 'DIG', 'DPM', 'DXG', 'DXS', 'EIB', 'EVF',
       'FPT', 'FRT', 'FTS', 'GAS', 'GEX', 'GMD', 'GVR', 'HAG', 'HCM',
       'HDB', 'HDC', 'HDG', 'HHV', 'HPG', 'HSG', 'HT1', 'IMP', 'KBC',
       'KDC', 'KDH', 'KOS', 'LPB', 'MBB', 'MSB', 'MSN', 'MWG', 'NKG',
       'NLG', 'NT2', 'NVL', 'OCB', 'PAN', 'PC1', 'PDR', 'PHR', 'PLX',
       'PNJ', 'POW', 'PPC', 'PTB', 'PVD', 'PVT', 'REE', 'SAB', 'SBT',
       'SCS', 'SHB', 'SIP', 'SJS', 'SSB', 'SSI', 'STB', 'SZC', 'TCB',
       'TCH', 'TLG', 'TPB', 'VCB', 'VCG', 'VCI', 'VGC', 'VHC', 'VHM',
       'VIB', 'VIC', 'VIX', 'VJC', 'VND', 'VNM', 'VPB', 'VPI', 'VRE',
       'VSH'], dtype=object)

In [ ]:
global_start = '2018-12-01'
global_end = '2024-09-30'

market_data = Vnstock().stock(symbol = 'VNINDEX', source = 'VCI').\
quote.history(start = global_start, end = global_end, interval = '1M')
market_data['profit'] = (market_data['close'] - market_data['close'].shift(1)) / market_data['close'].shift(1) * 100
market_data.dropna(inplace = True)
bond_data = pd.read_csv('Vietnam_10_year_Government_Bond.csv')
bond_data.columns = ['time', 'close', 'open',' high', 'low', 'change']
bond_data['time'] = pd.to_datetime(bond_data['time'])
bond_data['profit'] = ((1 + bond_data['close']/100) ** (1/12) - 1) * 100

2024-12-12 20:16:33 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [72]:
market_data

,time,open,high,low,close,volume
0,2018-12-31,937.70,966.83,882.17,892.54,2737262950
1,2019-01-31,900.10,920.36,861.85,910.65,2336924730
2,2019-02-28,911.98,999.90,907.52,965.47,2521514923
3,2019-03-31,971.05,1014.51,964.76,980.76,3650841840
4,2019-04-30,984.89,1000.02,959.33,979.64,2414442910
...,...,...,...,...,...,...
65,2024-05-31,1212.13,1285.63,1206.28,1261.72,19391349135
66,2024-06-30,1272.37,1306.01,1240.15,1245.32,18124606963
67,2024-07-31,1245.22,1297.96,1218.65,1251.51,15636641832
68,2024-08-31,1251.51,1290.45,1184.53,1283.87,15466873122


In [30]:
# vn100_tickers = stock.listing.symbols_by_group('VN30').values
# vn100_tickers

In [31]:
stock = Vnstock().stock(symbol = "VN30", source = 'VCI')
history = stock.quote.history(start = start, end = end, interval = '1M')
history

2024-12-12 13:06:19 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume
0,2018-12-31,896.50,935.33,850.33,854.99,933516480
1,2019-01-31,855.01,874.09,821.83,866.06,766764380
2,2019-02-28,866.01,944.01,859.38,904.98,884913030
3,2019-03-31,904.96,939.17,889.17,900.13,1044225870
4,2019-04-30,900.17,912.94,873.55,890.55,642738550
...,...,...,...,...,...,...
65,2024-05-31,1244.15,1322.13,1239.02,1275.97,5999064371
66,2024-06-30,1285.07,1339.17,1272.27,1278.32,6066253975
67,2024-07-31,1278.22,1324.91,1262.77,1299.09,5638379686
68,2024-08-31,1299.09,1333.56,1226.37,1331.52,5556648952


In [32]:
# quarter_start_months = {1: '01', 2: '04', 3: '07', 4: '10'}

# class Company:
#     def __init__(self, ticker, start = '2018-12-01', end = '2024-09-30'):
#         self.ticker = ticker
#         stock = Vnstock().stock(symbol = self.ticker, source = 'VCI')
#         self.history = stock.quote.history(start = start, end = end, interval = '1M')
        
#         self.history.loc[:, ['open', 'close', 'high', 'low']] = self.history[['open', 'close', 'high', 'low']] * 1000
#         self.ratio = stock.finance.ratio(period='quarter', lang='en', dropna=True)
#         self.balance = stock.finance.balance_sheet(period='quarter', lang='en', dropna=True)
#         self.history['shares'] = np.nan

#         # Tính market cap, và book value
#         meta = self.ratio['Meta']
#         meta['date'] = np.nan
#         stats = self.ratio['Chỉ tiêu định giá']
#         for i in meta.index:
#             meta.loc[i, 'date'] = f'{meta['yearReport'][i]}-{quarter_start_months[meta['lengthReport'][i]]}-01'
#         meta['date'] = pd.to_datetime(meta['date'])
#         self.balance['date'] = np.nan
#         for i in self.balance.index:
#             self.balance.loc[i, 'date'] = f'{self.balance['yearReport'][i]}-{quarter_start_months[self.balance['lengthReport'][i]]}-01'
#         self.balance.loc[:, 'date'] = pd.to_datetime(self.balance['date'])

#         for i in self.history.index:
#             target_date = (meta['date'] - self.history['time'][i]).abs().argmin()
#             self.history.loc[i, 'shares'] = stats['Outstanding Share (Mil. Shares)'][target_date]
#             self.history.loc[i, 'book_value'] = self.balance["OWNER'S EQUITY(Bn.VND)"][target_date]

#         self.history['market_cap'] = self.history['shares'] * self.history['close']

#         # Tính Book to Market Ratio
#         self.history.loc[:, 'btm'] = self.history['book_value'] / self.history['market_cap']

#         self.history.loc[:, 'hml'] = self.history['btm'].apply(lambda x : 'high' if x >= 1 else 'low')

#         self.history.loc[:, 'monthly_profit'] = (self.history['close'] - self.history['close'].shift(1)) / self.history['close'].shift(1) * 100

#         self.history = self.history.dropna()

In [ ]:
quarter_start_months = {1: '01', 2: '04', 3: '07', 4: '10'}

class Company:
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        stock = Vnstock().stock(symbol = self.ticker, source = 'VCI')
        self.history = stock.quote.history(start = start, end = end, interval = '1M')
        
        self.history.loc[:, ['open', 'close', 'high', 'low']] = self.history[['open', 'close', 'high', 'low']] * 1000
        self.ratio = stock.finance.ratio(period='quarter', lang='en', dropna=True)
        self.balance = stock.finance.balance_sheet(period='quarter', lang='en', dropna=True)
        self.history['shares'] = np.nan

        # Tính market cap, và book value
        meta = self.ratio['Meta']
        meta['date'] = np.nan
        stats = self.ratio['Chỉ tiêu định giá']
        for i in meta.index:
            meta.loc[i, 'date'] = f'{meta['yearReport'][i]}-{quarter_start_months[meta['lengthReport'][i]]}-01'
        meta['date'] = pd.to_datetime(meta['date'])
        self.balance['date'] = np.nan
        for i in self.balance.index:
            self.balance.loc[i, 'date'] = f'{self.balance['yearReport'][i]}-{quarter_start_months[self.balance['lengthReport'][i]]}-01'
        self.balance.loc[:, 'date'] = pd.to_datetime(self.balance['date'])

        for date in meta['date'].index:
            if (meta.loc[date, 'date'] < pd.to_datetime(end)) and (meta.loc[date, 'date'] > pd.to_datetime(start)):
                temp_day = 0
                while True:
                    if len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    elif len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    temp_day += 1
        self.history = self.history.fillna(method = 'ffill').fillna(method = 'bfill')
        self.history['market_cap'] = self.history['shares'] * self.history['close']

        # Tính Book to Market Ratio
        self.history.loc[:, 'btm'] = self.history['book_value'] / self.history['market_cap']

        self.history.loc[:, 'hml'] = self.history['btm'].apply(lambda x : 'high' if x >= 1 else 'low')

        self.history.loc[:, 'monthly_profit'] = (self.history['close'] - self.history['close'].shift(1)) / self.history['close'].shift(1) * 100

        self.history = self.history.iloc[1:]

In [ ]:

class Portfolio:
    def __init__(self, tickers, start, end):
        self.tickers = tickers  
        self.data = {}
        self.betas = []
        self.start = start
        self.end = end
        
    def create_profiles(self):
        for ticker in self.tickers:
            print(ticker)
            self.data[ticker] = Company(ticker, start = self.start, end = self.end)
        for ticker in self.tickers:
            self.data[ticker].history['smb'] = np.nan

    def create_sub_portfolio(self, sub_tickers):
        new_portfolio = Portfolio(tickers = sub_tickers)
        for ticker in sub_tickers:
            new_portfolio.data[ticker] = copy.deepcopy(self.data[ticker])
        return new_portfolio
    
    def evaluate_smb(self):
        aggregate_market_cap = pd.DataFrame([], columns= self.tickers)

        for index in self.data[self.tickers[0]].history.index:
            for ticker in self.tickers: 
                aggregate_market_cap.loc[index, ticker] = self.data[ticker].history.loc[index, 'market_cap']
                
        for index in aggregate_market_cap.index:
            percentiles = np.percentile(aggregate_market_cap.loc[index].values, [10, 30]) 
            for ticker in self.tickers:
                x = aggregate_market_cap.loc[index, ticker] 
                if x <= percentiles[0]:
                    self.data[ticker].history.loc[index, 'smb'] = 'small' 
                elif x <= percentiles[1]:
                    self.data[ticker].history.loc[index, 'smb'] = 'mid'
                else:
                    self.data[ticker].history.loc[index, 'smb'] = 'big'

    def setup_regression(self, market_data, bond_data):
        regression_data = pd.DataFrame([], columns= ['time', 'excess_return', 'mp', 'smb', 'hml'])

        for index in self.data[self.tickers[0]].history.index:
            date = self.data[self.tickers[0]].history.loc[index, 'time']
            smb = 0
            hml = 0

            temp_day = 0
            
            while True:
                try:         # Because sometimes the date is off by 1 or 2 days
                    rf = bond_data[bond_data['time'] == date + timedelta(days = temp_day)]['profit'].values[0]
                    break
                except:
                    temp_day -= 1
                    
            mp = market_data.loc[index, 'profit'] - rf

            for ticker in self.tickers:
                if self.data[ticker].history.loc[index, 'smb'] == 'small':
                    smb += self.data[ticker].history.loc[index, 'monthly_profit']
                elif self.data[ticker].history.loc[index, 'smb'] == 'big':
                    smb -= self.data[ticker].history.loc[index, 'monthly_profit']
                if self.data[ticker].history.loc[index, 'hml'] == 'high':
                    hml += self.data[ticker].history.loc[index, 'monthly_profit']
                else:
                    hml -= self.data[ticker].history.loc[index, 'monthly_profit']
            
            regression_data.loc[index, 'smb'] = smb
            regression_data.loc[index, 'hml'] = hml
            regression_data.loc[index, 'mp'] = mp
            regression_data.loc[index, 'rf'] = rf
            regression_data.loc[index, 'time'] = date
        # regression_data['excess_return'] = regression_data['excess_return'].shift(-1)

        # regression_data.dropna(inplace = True)
        return regression_data
        
    def ff3_metrics(self, og_regression_data, use_data = 'newest', cutoff = None):
        # TÍNH EXCESS RETURN CHO TỪNG PORTFOLIO LẺ
        regression_data = copy.deepcopy(og_regression_data)

        for index in regression_data.index:
            total_return = np.mean([self.data[ticker].history.loc[index, 'monthly_profit'] for ticker in self.tickers]) - regression_data.loc[index, 'rf']
            regression_data.loc[index, 'excess_return'] = total_return

        if cutoff is None:
            cutoff = round((len(regression_data) * 60/100))
        
        train_set = regression_data[:cutoff]
        test_set = regression_data[cutoff:]

        y_train = train_set['excess_return']
        
        x_train = train_set.drop(columns = ['time', 'excess_return', 'rf'])

        y_test = test_set['excess_return']
        if use_data == 'newest':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])

        elif use_data == 'average':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])
            for column in x_test.columns:
                m = np.mean(x_train[column])
                for index in x_test.index:
                    x_test.loc[index, column] = m

        model = LinearRegression()
        # model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)

        model.fit(x_train, y_train)
        
        y_train_pred = model.predict(x_train)
        
        mse_train = mean_squared_error(y_train, y_train_pred)

        y_pred = model.predict(x_test)
        mse_test = mean_squared_error(y_test, y_pred)
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_pred)

        return mse_train, mse_test, r2_train, r2_test, model.coef_

In [35]:
# full_portfolio = Portfolio(['FPT', 'VNM', 'ACB'])
full_portfolio = Portfolio(vn30_tickers)

full_portfolio.create_profiles()

ACB


2024-12-12 13:06:20 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


BCM


2024-12-12 13:06:22 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


BID


...

2024-12-12 13:11:20 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


VPB


2024-12-12 13:11:35 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


VRE


2024-12-12 13:11:49 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [36]:
aggregate_market_cap = full_portfolio.evaluate_smb()

In [37]:
regression_data = full_portfolio.setup_regression(market_data, bond_data)

In [38]:
mse_train, mse_test, r2_train, r2_test, _ = full_portfolio.ff3_metrics(regression_data, use_data = 'newest')
print(f'MSE train: {mse_train}')
print(f'R2 train: {r2_train}')

print(f'MSE test: {mse_test}')
print(f'R2 test: {r2_test}')

MSE train: 2.4503437114083204
R2 train: 0.9582625448063652
MSE test: 1.2600486992393272
R2 test: 0.9688578378471513


In [39]:
cutoff = round((len(regression_data) * 60/100))

In [40]:
price_df = pd.DataFrame([], columns= vn30_tickers, index = full_portfolio.data[vn30_tickers[0]].history.index)
for index in price_df.index:
    for ticker in vn30_tickers:
        price_df.loc[index, ticker] = full_portfolio.data[ticker].history.loc[index, 'monthly_profit']
price_df['average_monthly_profit'] = price_df.mean(axis = 1)
price_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time']
vn30_market_growth = 1
for index in price_df[cutoff:].index:
    vn30_market_growth = vn30_market_growth * (1 + price_df.loc[index, 'average_monthly_profit'] / 100)
vn30_market_growth = (vn30_market_growth - 1) * 100
vn30_market_growth

15.049826240220664

In [41]:
results = []

for ticker in vn30_tickers:
    temp = full_portfolio.create_sub_portfolio(sub_tickers= [ticker])
    
    _,_,_,_, [mp, smb, hml] = temp.ff3_metrics(regression_data)
    results.append([ticker, mp, smb, hml])

    del temp


In [42]:
mp_results = sorted(results, key = lambda x : x[1])
smb_results = sorted(results, key = lambda x : x[2])
hml_results = sorted(results, key = lambda x : x[3])


In [43]:
import itertools

strategies = [list(i) for i in list(itertools.product([0, 1], repeat=3))]

In [ ]:
portfolios = []
for strategy in strategies:
    portfolio = []
    if strategy[0] == 0:
        portfolio.extend([i[0] for i in mp_results][:2])
    else:
        portfolio.extend([i[0] for i in mp_results[::-1]][:2])
    if strategy[1] == 0:
        portfolio.extend([i[0] for i in smb_results][:2])
    else:
        portfolio.extend([i[0] for i in smb_results[::-1]][:2])

    if strategy[2] == 0:
        portfolio.extend([i[0] for i in hml_results][:2])
    else:
        portfolio.extend([i[0] for i in hml_results[::-1]][:2])
    portfolios.append(list(set(portfolio)))

In [45]:
portfolios

[['TPB', 'BCM', 'GAS', 'MBB'],
 ['BCM', 'GAS', 'MWG', 'POW'],
 ['TPB', 'BCM', 'GAS', 'SSI', 'VIB', 'MBB'],
 ['POW', 'MWG', 'BCM', 'GAS', 'SSI', 'VIB'],
 ['TPB', 'GVR', 'BCM', 'GAS', 'SSI', 'MBB'],
 ['GVR', 'POW', 'MWG', 'BCM', 'GAS', 'SSI'],
 ['TPB', 'GVR', 'SSI', 'VIB', 'MBB'],
 ['GVR', 'POW', 'MWG', 'SSI', 'VIB']]

In [46]:
test_data = pd.DataFrame([], columns = vn30_tickers)
for index in full_portfolio.data[vn30_tickers[0]].history.index:
    for ticker in vn30_tickers:
        test_data.loc[index, ticker] = full_portfolio.data[ticker].history.loc[index, 'monthly_profit']
test_data = test_data.iloc[cutoff:]

In [47]:
def backtest_basic(data, portfolio):
    temp_df = pd.DataFrame([], columns = portfolio)
    for index in data.index:
        for ticker in portfolio:
            temp_df.loc[index, ticker] = data.loc[index, ticker]
    temp_df['portfolio_profit'] = temp_df.mean(axis = 1)
    temp_df['market_growth'] = data.mean(axis = 1)
    temp_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time'].iloc[cutoff:]
    return temp_df

backtest_results = {}
for portfolio in portfolios:
    backtest_results[tuple(portfolio)] = backtest_basic(test_data, portfolio)

metrics_df = pd.DataFrame(list(zip(strategies, portfolios)), columns = ['strategy', 'portfolio'])

metrics_df['ap'] = 0
metrics_df['al'] = 0
metrics_df['np'] = 0
metrics_df['wr'] = 0

metrics_df['market_profit'] = 0

# Average profit, average loss, net profit, win rate

for portfolio in portfolios:
    backtest_result = backtest_results[tuple(portfolio)]
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'ap'] = backtest_result.loc[backtest_result['portfolio_profit'] > 0, 'portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'al'] = backtest_result.loc[backtest_result['portfolio_profit'] < 0, 'portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'np'] = backtest_result['portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'wr'] = len(backtest_result.loc[backtest_result['portfolio_profit'] > 0, 'portfolio_profit']) / len(backtest_result) * 100

metrics_df['market_profit'] = len(metrics_df) * [backtest_results[tuple(portfolios[0])]['market_growth'].mean()]

In [49]:
metrics_df

,strategy,portfolio,ap,al,np,wr,market_profit
0,"[0, 0, 0]","[TPB, BCM, GAS, MBB]",4.315067,-6.469168,0.463554,64.285714,0.705835
1,"[0, 0, 1]","[BCM, GAS, MWG, POW]",5.146694,-5.460663,0.221850,53.571429,0.705835
2,"[0, 1, 0]","[TPB, BCM, GAS, SSI, VIB, MBB]",5.520860,-7.643193,0.819413,64.285714,0.705835
3,"[0, 1, 1]","[POW, MWG, BCM, GAS, SSI, VIB]",5.207519,-7.530361,0.658276,64.285714,0.705835
4,"[1, 0, 0]","[TPB, GVR, BCM, GAS, SSI, MBB]",6.143531,-8.117920,1.050156,64.285714,0.705835
5,"[1, 0, 1]","[GVR, POW, MWG, BCM, GAS, SSI]",6.218079,-7.346799,0.889020,60.714286,0.705835
6,"[1, 1, 0]","[TPB, GVR, SSI, VIB, MBB]",7.156936,-10.613061,1.445151,67.857143,0.705835
7,"[1, 1, 1]","[GVR, POW, MWG, SSI, VIB]",7.179302,-9.417739,1.251787,64.285714,0.705835


In [50]:
# Explanation of coefficients:

# MP: Higher beta is good in bullish markets, lower beta is good in bearish market.
# SMB: Higher is more risky. Lower is safer
# HML: Higher is good in economic recoveries or in mature markets. Lower is good in strong bull market.

# 1 is high, 0 is low

In [51]:
# - An adaptive algorithm utilizing the Fama French 3 model for improving portfolios
# - 

In [ ]:
def total_profit(history):
    for index in history.index:
        total_profit = (1 + history.loc[index, 'profit']/100)
    return (total_profit - 1) * 100

In [53]:
def backtest_single(data, portfolio, log = False):
    budget_single = 10000000
    temp_df = pd.DataFrame([], columns = portfolio)

    for index in data.index:
        for ticker in portfolio:
            temp_df.loc[index, ticker] = data.loc[index, ticker]
    temp_df['portfolio_profit'] = temp_df.mean(axis = 1)
    temp_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time'].iloc[cutoff:]

    for index in temp_df.index:
        budget_single = budget_single * (1 + temp_df.loc[index, 'portfolio_profit'] / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {temp_df.loc[index, 'portfolio_profit']}%    Current total assets: {budget_single}")
    profit_single  = ((budget_single - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_single}')
    return temp_df, profit_single

def backtest_diverse(data, portfolios, log = False):
    temp_dfs = []
    budget = 10000000

    history = pd.DataFrame([], columns = ['date', 'profit'])

    for portfolio in portfolios:
        temp_df, _ = backtest_single(data, portfolio)
        temp_dfs.append(temp_df)
    for index in temp_df.index:
        profit_month = np.mean([i.loc[index, 'portfolio_profit'] for i in temp_dfs])
        budget = budget * (1 + profit_month / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {profit_month}%    Current total assets: {budget}")
        
        history.loc[index, 'profit'] = profit_month

    profit_full  = ((budget - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_full}')
    
    metrics = pd.DataFrame([], columns = ['ap', 'al', 'np', 'wr'])
    metrics['ap'] = history[history['profit'] > 0].mean()
    metrics['al'] = history[history['profit'] < 0].mean()
    metrics['np'] = history['profit'].mean()
    metrics['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = temp_df['date'].reset_index(drop = True)
    metrics = metrics.iloc[1].to_frame()
    
    return history, metrics

In [54]:
history, metrics = backtest_diverse(test_data, portfolios)

In [55]:
metrics

,profit
ap,5.639423
al,-7.771238
np,0.849901
wr,0.642857


In [56]:
total_profit(history)

np.float64(16.913446247873765)

In [62]:
portfolios

[['TPB', 'BCM', 'GAS', 'MBB'],
 ['BCM', 'GAS', 'MWG', 'POW'],
 ['TPB', 'BCM', 'GAS', 'SSI', 'VIB', 'MBB'],
 ['POW', 'MWG', 'BCM', 'GAS', 'SSI', 'VIB'],
 ['TPB', 'GVR', 'BCM', 'GAS', 'SSI', 'MBB'],
 ['GVR', 'POW', 'MWG', 'BCM', 'GAS', 'SSI'],
 ['TPB', 'GVR', 'SSI', 'VIB', 'MBB'],
 ['GVR', 'POW', 'MWG', 'SSI', 'VIB']]

In [57]:
def backtest_adaptive(data, portfolios, log = False):
    budget = 10000000
    temp_dfs = []

    for portfolio in portfolios:
        temp_df, _ = backtest_single(data, portfolio)
        temp_dfs.append([portfolio, temp_df])
    
    history = pd.DataFrame([], columns = ['date', 'profit'])
    chosen_portfolio = sorted(temp_dfs, key = lambda x : x[1].iloc[0]['portfolio_profit'], reverse= True)[0][0] 
    
    for i in range(1, len(data[1:])):
        profit = data.iloc[i][chosen_portfolio].mean()
        budget = budget * (1 + profit / 100)
        if log:
            print(f"Date: {temp_df.iloc[i]['date']}    Portfolio: {chosen_portfolio}    Portfolio changed by: {profit}%    Current total assets: {budget}")
        chosen_portfolio = sorted(temp_dfs, key = lambda x : x[1].iloc[i]['portfolio_profit'], reverse= True)[0][0] 
        history.loc[i, 'profit'] = profit
    if log:
        print(f'Trading finished. Total profit: {(budget - 10000000) / 10000000 * 100}')

    metrics = pd.DataFrame([], columns = ['ap', 'al', 'np', 'wr'])
    metrics['ap'] = history[history['profit'] > 0].mean()
    metrics['al'] = history[history['profit'] < 0].mean()
    metrics['np'] = history['profit'].mean()
    metrics['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = temp_df['date'].reset_index(drop = True)
    metrics = metrics.iloc[1].to_frame()
    return history, metrics

In [66]:
backtest_single(test_data, ['FPT', 'GAS'])

(          FPT        GAS portfolio_profit       date
 42  -5.017592  -1.860465        -3.439029 2022-06-30
 43  -3.140602  -4.399076        -3.769839 2022-07-31
 44   4.921849   8.936062         6.928956 2022-08-31
 45   -7.03645  -6.067678        -6.552064 2022-09-30
 46  -6.085919   1.006211        -2.539854 2022-10-31
 47  -1.325104  -0.811708        -1.068406 2022-11-30
 48   3.090508  -7.898326        -2.403909 2022-12-31
 49    8.56531   4.725363         6.645337 2023-01-31
 50  -3.583169  -2.622445        -3.102807 2023-02-28
 51  -1.738834  -1.452145         -1.59549 2023-03-31
 52   -2.02984  -9.028801        -5.529321 2023-04-30
 53   8.517797        0.0         4.258899 2023-05-31
 54   2.268277   0.220881         1.244579 2023-06-30
 55  15.781075   9.241845         12.51146 2023-07-31
 56  14.291621   0.564896         7.428258 2023-08-31
 57  -4.027493  10.391868         3.182188 2023-09-30
 58 -10.566654 -15.543979       -13.055316 2023-10-31
 59  10.733352  -0.143451   

In [64]:
history, metrics = backtest_adaptive(test_data, portfolios, log = True)

Date: 2022-07-31 00:00:00    Portfolio: ['BCM', 'GAS', 'MWG', 'POW']    Portfolio changed by: -2.804556821190993%    Current total assets: 9719544.3178809
Date: 2022-08-31 00:00:00    Portfolio: ['TPB', 'GVR', 'SSI', 'VIB', 'MBB']    Portfolio changed by: 6.613927066174645%    Current total assets: 10362387.890230065
Date: 2022-09-30 00:00:00    Portfolio: ['BCM', 'GAS', 'MWG', 'POW']    Portfolio changed by: -8.22328745396753%    Current total assets: 9510258.946921326
Date: 2022-10-31 00:00:00    Portfolio: ['BCM', 'GAS', 'MWG', 'POW']    Portfolio changed by: -10.885783328962631%    Current total assets: 8474992.763936188
Date: 2022-11-30 00:00:00    Portfolio: ['TPB', 'BCM', 'GAS', 'MBB']    Portfolio changed by: 0.6273360072102251%    Current total assets: 8528159.445152821
Date: 2022-12-31 00:00:00    Portfolio: ['TPB', 'GVR', 'SSI', 'VIB', 'MBB']    Portfolio changed by: -4.62016962507589%    Current total assets: 8134144.01288983
Date: 2023-01-31 00:00:00    Portfolio: ['TPB', 

In [59]:
metrics

,profit
ap,5.857344
al,-7.86263
np,1.635813
wr,0.692308


In [60]:
total_profit(history)

np.float64(42.04242224512085)

In [61]:
regression_data[cutoff:]

,time,excess_return,mp,smb,hml,rf
42,2022-06-30 00:00:00,NaN,-7.631687,52.390626,5.178852,0.276425
43,2022-07-31 00:00:00,NaN,0.431216,-3.954981,-59.435625,0.297742
44,2022-08-31 00:00:00,NaN,5.847536,-116.466936,-126.810978,0.301693
45,2022-09-30 00:00:00,NaN,-11.99782,169.223225,62.237407,0.408687
46,2022-10-31 00:00:00,NaN,-9.624262,104.023908,-64.025647,0.422859
47,2022-11-30 00:00:00,NaN,1.571862,-105.910262,-67.618915,0.420472
48,2022-12-31 00:00:00,NaN,-4.352483,66.037685,21.919911,0.410360
49,2023-01-31 00:00:00,NaN,9.95712,-179.95229,-196.467941,0.378599
50,2023-02-28 00:00:00,NaN,-8.154559,137.08863,81.869318,0.370042
51,2023-03-31 00:00:00,NaN,3.615168,-62.4778,-61.224336,0.284586
